In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-recomedation-system/final_data_to_work(1).pkl


In [3]:
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df1 = pd.read_pickle('/kaggle/input/food-recomedation-system/final_data_to_work(1).pkl')
df1

,user_id,recipe_id,date,rating,review
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
5,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...
8,76535,134728,2005-09-02,4,Very good!
...,...,...,...,...,...
1131780,113509,130019,2012-04-20,2,Hubby and I thought this was good. Neither of ...
1131997,51224,287006,2009-01-22,2,This is the recipe that is all over the intern...
1132031,811179,136702,2011-12-29,2,We made our gluten-free with GF flour and it w...
1132282,176615,118119,2006-10-27,2,We've tried lots of peanut butter pasta recipe...


In [5]:
n_users = len(df1.user_id.unique())
n_recipe = len(df1.recipe_id.unique())
print('Unique Users: {0}, unique recipe: {1}'.format(n_users, n_recipe))

Unique Users: 3402, unique recipe: 189182


In [6]:
user_item_m = df1.pivot('user_id','recipe_id','rating').fillna(0)
user_item_m

recipe_id,38,40,41,45,46,49,52,53,55,58,...,536843,536990,537071,537073,537175,537241,537319,537458,537459,537485
user_id,,,,,,,,,,,,,,,,,,,,,
1533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001297534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2001330613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001362355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Matrix Factorization using inbuilt library

In [6]:
R = np.array(user_item_m)
user_ratings_mean = np.mean(R, axis = 1)

In [7]:
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [8]:
from scipy import linalg                                                                            
from scipy.sparse import linalg as slinalg      
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [9]:
U.shape

(3402, 50)

In [10]:
sigma.shape

(50,)

In [11]:
s = np.diag(sigma)

In [12]:
s.shape

(50, 50)

In [13]:
Vt.shape

(50, 189182)

In [14]:
all_user_predicted_ratings = np.dot(np.dot(U, s), Vt) + user_ratings_mean.reshape(-1, 1)
preds_rating = pd.DataFrame(all_user_predicted_ratings, columns = user_item_m.columns)

In [15]:
preds_rating

recipe_id,38,40,41,45,46,49,52,53,55,58,...,536843,536990,537071,537073,537175,537241,537319,537458,537459,537485
0,0.002839,0.004663,0.003041,0.002834,0.003644,0.003527,0.001376,0.002833,0.002681,0.023725,...,0.002791,-0.000133,0.002863,0.002883,-0.000815,0.001950,0.002611,0.002721,0.002721,0.002792
1,0.011343,0.023242,0.013453,0.013623,0.024752,-0.054878,0.024844,0.009551,0.014419,0.003520,...,0.008548,0.029210,0.010519,0.011042,0.016034,0.009017,0.007337,0.013583,0.013583,0.008758
2,0.001051,0.001847,0.001061,0.001030,0.001567,-0.001818,0.001979,0.000982,0.000985,0.016995,...,0.000969,-0.000382,0.000994,0.001001,-0.000083,0.000324,0.000825,0.003376,0.003376,0.001003
3,0.001014,0.001136,0.000976,0.001055,0.000930,0.000431,0.003357,0.000938,0.000949,0.004963,...,0.000947,0.001411,0.000961,0.000965,0.000575,0.000659,0.000923,0.000290,0.000290,0.000959
4,0.006833,0.010812,0.007329,0.007270,0.008433,0.042823,0.035271,0.006475,0.006689,0.020090,...,0.006376,-0.000177,0.006794,0.006906,-0.013573,0.002286,0.006225,0.001731,0.001731,0.006440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,0.001229,0.000772,0.001143,0.001228,0.001522,0.000504,0.007869,0.000802,0.001299,0.003823,...,0.000805,0.008775,0.000858,0.000873,-0.005172,0.003253,0.000881,0.007242,0.007242,0.001022
3398,0.000138,0.000723,-0.000014,0.000386,0.001014,0.005469,-0.005233,-0.000410,0.000244,0.007548,...,-0.000496,0.004945,-0.000136,-0.000040,-0.004078,-0.002847,-0.000369,0.002794,0.002794,-0.000434
3399,0.001234,0.000254,0.000979,0.001818,0.002571,0.013441,-0.005263,0.000423,0.001780,-0.002865,...,0.000364,0.002808,0.000842,0.000971,0.003925,-0.003605,0.000348,-0.001448,-0.001448,0.000497
3400,0.000850,0.000925,0.000845,0.000830,0.000900,-0.002148,0.003226,0.000831,0.000865,-0.000873,...,0.000810,-0.000756,0.000842,0.000850,-0.000045,-0.000192,0.000795,-0.000491,-0.000491,0.000821


In [16]:
def recommend():
    n = int(input("Enter User ID : "))
    a = preds_rating.iloc[n,:]
    a = pd.DataFrame(a)
    a['recipe_id'] = user_item_m.columns
    a = a.reset_index(drop=True)
    name = a.columns
    name = name[0]
    a = a.rename(columns = {name:'id'})
    a = a.sort_values(by='id',ascending=[False])
    print(list(a.iloc[:10,1]))

In [17]:
recommend()

Enter User ID :  1


[77397, 27208, 25885, 28148, 32204, 89204, 29679, 95222, 22782, 39087]


### Matrix Factorization from scratch

In [7]:
K = 50
eta = 0.001
lambda1 = 0.7
iterations = 20

In [8]:
U = np.random.normal( 1000, 1000 ,size=(n_users, K))
V = np.random.normal( 1000, 1000 ,size=(n_recipe, K))

In [9]:
U

array([[ 1792.42657988,  1440.57457509,  1970.22705534, ...,
          385.91149908,   664.42974795,   738.65724824],
       [ -109.24935084,  2063.75206861,  1363.21439801, ...,
         1238.48613841,  1277.32301832,   149.58087109],
       [-1509.37815696,  2192.09400806,  1017.85860646, ...,
          758.28188185,  -606.08654478,  2930.71840366],
       ...,
       [  553.02036856,  2177.11033748,  1820.26954526, ...,
         1141.51365937,  2096.54027287,  1147.42911   ],
       [  733.43672786,  2857.01159931, -1350.74378606, ...,
         1528.56519113,  2668.45526429,   757.02605701],
       [ -189.28852285,   335.97044405,   880.43678614, ...,
          829.07493088,  3103.0512037 ,   806.35403319]])

In [10]:
V

array([[  919.29615035,  2088.53866425,  1551.19951824, ...,
          839.54695369,  1257.31553754,   893.65877309],
       [ 1245.63612068,   597.95851651,  2220.56755747, ...,
          129.10298343,   -34.19692029,  1143.4724488 ],
       [ 1324.56374063,   516.12599718,  1394.39218249, ...,
          711.17083447,  1386.24726028,  2213.69023593],
       ...,
       [  893.01488479,  2212.24550161,  -210.41421809, ...,
         1256.91104855, -1174.16257726,  1907.97367988],
       [ 1686.76442753,   845.4963704 ,  1124.76629794, ...,
         1140.0245619 ,  2409.5683827 ,  -450.39615984],
       [ 1722.85239451,  2348.16528558,  2246.93920248, ...,
         2332.40922946,  1474.79324868,  1938.64462994]])

In [11]:
print("Shape of U : ",U.shape)
print("Shape of V : ",V.shape)

Shape of U :  (3402, 50)
Shape of V :  (189182, 50)


In [12]:
for i in range(0,1):
    UV_T = np.dot(U,V.T)
    R_UV_T = np.subtract(user_item_m,UV_T)
    
    U = U + eta * (  (np.dot(R_UV_T,-V))    + (lambda1*U)  )
    V = V + eta * (  (np.dot(R_UV_T.T,-U))  + (lambda1*V)  )
    
    eta = eta / (eta + 0.4)

In [13]:
U

array([[1.18617192e+13, 1.17858642e+13, 1.18687728e+13, ...,
        1.15852441e+13, 1.16049593e+13, 1.16653287e+13],
       [8.58056405e+12, 8.98488275e+12, 8.84025789e+12, ...,
        8.82761918e+12, 8.81631866e+12, 8.63049542e+12],
       [7.93928045e+12, 8.63625535e+12, 8.40986794e+12, ...,
        8.36624718e+12, 8.08547042e+12, 8.79298780e+12],
       ...,
       [9.79291242e+12, 1.00967368e+13, 1.00148410e+13, ...,
        9.89435098e+12, 1.00536280e+13, 9.90836283e+12],
       [9.62560367e+12, 1.00221793e+13, 9.22050607e+12, ...,
        9.76948536e+12, 9.96400507e+12, 9.63214151e+12],
       [6.45675129e+12, 6.54952576e+12, 6.64427170e+12, ...,
        6.64189226e+12, 7.06356232e+12, 6.64083550e+12]])

In [14]:
V

array([[1.36597389e+21, 1.36577410e+21, 1.36419103e+21, ...,
        1.36478699e+21, 1.36280833e+21, 1.36675430e+21],
       [1.40757382e+21, 1.40639827e+21, 1.40612856e+21, ...,
        1.40582491e+21, 1.40317938e+21, 1.40829545e+21],
       [1.82812947e+21, 1.82644665e+21, 1.82548731e+21, ...,
        1.82642650e+21, 1.82368608e+21, 1.82994933e+21],
       ...,
       [1.65218238e+21, 1.65188336e+21, 1.64884443e+21, ...,
        1.65119677e+21, 1.64660681e+21, 1.65396176e+21],
       [1.60251186e+21, 1.60071972e+21, 1.59972086e+21, ...,
        1.60079256e+21, 1.59893024e+21, 1.60205422e+21],
       [1.94375424e+21, 1.94289798e+21, 1.94093758e+21, ...,
        1.94252850e+21, 1.93862831e+21, 1.94497927e+21]])

In [15]:
preds_rating  = np.dot(U,V.T)
preds_rating

array([[8.00656424e+35, 8.24944005e+35, 1.07157057e+36, ...,
        9.68540941e+35, 9.39053988e+35, 1.13912522e+36],
       [5.97575716e+35, 6.15700823e+35, 7.99775629e+35, ...,
        7.22877763e+35, 7.00870014e+35, 8.50194887e+35],
       [5.72052527e+35, 5.89403886e+35, 7.65612890e+35, ...,
        6.92004697e+35, 6.70931685e+35, 8.13880433e+35],
       ...,
       [6.73377769e+35, 6.93801466e+35, 9.01224645e+35, ...,
        8.14572061e+35, 7.89771311e+35, 9.58040129e+35],
       [6.59545381e+35, 6.79550634e+35, 8.82713969e+35, ...,
        7.97843667e+35, 7.73551070e+35, 9.38358822e+35],
       [4.51013676e+35, 4.64692519e+35, 6.03621649e+35, ...,
        5.45582946e+35, 5.28972344e+35, 6.41672998e+35]])

In [20]:
def recommend():
    n = int(input("Enter User ID : "))
    a = preds_rating[n,:]
    a = pd.DataFrame(a)
    a['recipe_id'] = user_item_m.columns
    a = a.reset_index(drop=True)
    name = a.columns
    name = name[0]
    a = a.rename(columns = {name:'id'})
    a = a.sort_values(by='id',ascending=[False])
    print(list(a.iloc[:10,1]))

In [21]:
recommend()

Enter User ID :  1


[127680, 26276, 425314, 481315, 107546, 95171, 402465, 45603, 438944, 59410]
